In [1]:
import pandas as pd
from bs4 import BeautifulSoup
from urllib.request import urlopen
import webbrowser
import requests
import re
import datetime
import sys

In [2]:
auth_key="a2f0104fc81d7dc05f8817768aae6c55169516a6" #authority key
company_codes = pd.read_excel('상장법인목록.xlsx',converters={'종목코드':str})
company_codes1 = company_codes.loc[:,"종목코드"]
start_date="20140101"

In [3]:
### progressBar
def progressBar(value, endvalue, bar_length=20):
    percent = float(value) / endvalue
    arrow = '-' * int(round(percent * bar_length)-1) + '>'
    spaces = ' ' * (bar_length - len(arrow))

    sys.stdout.write("\rPercent: [{0}] {1}%".format(arrow + spaces, int(round(percent * 100))))
    sys.stdout.flush()

In [4]:
# 접수번호(rcp_no)에 해당하는 모든 하위 보고서 URL을 추출하여 리스트로 반환하는 함수
def get_sub_report_urls(rcp_no):
    doc_urls = []
    url = "http://dart.fss.or.kr/dsaf001/main.do?rcpNo=%s" % (rcp_no)
    r = requests.get(url)
    reg = re.compile('viewDoc\((.*)\);')
    params = []
    matches = reg.findall(r.text)
    for m in matches: 
        params.append(m.replace("'", "").replace(" ", "").split(","))
   
    doc_url_tmpl = "http://dart.fss.or.kr/report/viewer.do?rcpNo=%s&dcmNo=%s&eleId=%s&offset=%s&length=%s&dtd=%s" 

    for p in params:
        if rcp_no == p[0]:
            doc_urls.append( doc_url_tmpl % tuple(p) )
        
    return doc_urls

In [5]:
# 해당 url에서 텍스트를 가져오는 함수
def get_text_urls(rcp_no) :
    url = get_sub_report_urls(rcp_no)
    idx = 0
    while True:
        if len(url) == 0 or idx == len(url): 
            html_text = ''
            break
        url2 = url[idx]
        r = requests.get(url2)
        html_text = r.text
        # 이사의 경영진단 및 분석의견이 아닌 글은 다시 찾기
        if "V. 이사의 경영진단 및 분석의견" in html_text: 
            break
        else : idx += 1

    return html_text

In [6]:
dat = pd.read_excel("dart_crp_list.xlsx")
dat.head()

,crp_cls,crp_nm,crp_cd,rpt_nm,rcp_no,flr_nm,rcp_dt,rmk
0,N,엘에이티,311060,사업보고서 (2017.12),20190102000129,엘에이티,20190102,NaN
0,N,대원모방,311840,사업보고서 (2017.12),20181228000183,대원모방,20181228,NaN
0,N,지놈앤컴퍼니,314130,사업보고서 (2017.12),20181231000061,지놈앤컴퍼니,20181231,NaN
0,N,테크엔,308700,사업보고서 (2017.12),20181224000149,테크엔,20181224,NaN
0,N,프로테옴텍,303360,[기재정정]사업보고서 (2017.12),20181226000186,프로테옴텍,20181226,NaN


In [7]:
txt = pd.DataFrame(columns = ["회사명","회사코드","접수날짜","보고서 명","접수번호","텍스트"])
crp_lists = dat[['crp_nm','crp_cd','rcp_dt','rpt_nm','rcp_no']].values.tolist()
crp_lists[0]

['엘에이티', 311060, 20190102, '사업보고서 (2017.12)', 20190102000129]

In [8]:
################ 크롤링 ###########################################
start = datetime.datetime.now()

for idx in range(0, 10):
    crp = crp_lists[idx]
    html_txt = get_text_urls(crp[4])
    txt2 = pd.Series([crp[0],crp[1],crp[2],crp[3],crp[4],html_txt], 
                     index = ["회사명","회사코드","접수날짜","보고서 명","접수번호","텍스트"])
    txt = txt.append(txt2 , ignore_index = True)
    
    progressBar(idx, 10)

end = datetime.datetime.now()
print("\n걸린시간 : ",end-start)

ConnectionError: HTTPConnectionPool(host='dart.fss.or.kr', port=80): Max retries exceeded with url: /dsaf001/main.do?rcpNo=20190102000129 (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x000001EA7E6EFE10>: Failed to establish a new connection: [WinError 10060] 연결된 구성원으로부터 응답이 없어 연결하지 못했거나, 호스트로부터 응답이 없어 연결이 끊어졌습니다'))

In [19]:
txt.head()

,회사명,회사코드,접수날짜,보고서 명,접수번호,텍스트
0,엘에이티,311060,20190102,사업보고서 (2017.12),20190102000129,
1,엘에이티,311060,20190102,사업보고서 (2017.12),20190102000129,
2,대원모방,311840,20181228,사업보고서 (2017.12),20181228000183,
3,지놈앤컴퍼니,314130,20181231,사업보고서 (2017.12),20181231000061,
4,테크엔,308700,20181224,사업보고서 (2017.12),20181224000149,


In [18]:
############
writer = pd.ExcelWriter('./dart_text_1.xlsx')
txt.to_excel(writer,'Sheet1')
writer.save()